In [ ]:
import torch
import yaml
import random
import pandas
from google.colab import files
import numpy as np

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5  # clone repo
# !pip install -U -r yolov5/requirements.txt  # install dependencies
!pip install -r yolov5/requirements.txt  # install dependencies

In [ ]:
# best practice is to put dataset one directory above YOLO
%%capture
%cd /content/
# download dataset from roboflow

!curl -L "https://app.roboflow.com/ds/{url}" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip # Full 960 with Nulls

# roboflow wrote this yaml
# %cat data.yaml

In [ ]:
%%capture
%cd /content/yolov5

In [ ]:
# Train YOLOv5s
%time
# !python train.py --img {int} --batch {int} --epochs {int} --data {/path/data.yaml} --weights {yolov5s.pt} --cache

#
# 320 x 240 @ 10 FPS
# --img: image dimension (square)
# epochs should probably be 100+ in final depending on if it overfits?
# what's the best model size for maximizing speed/accuracy?

!python train.py --img 960 --batch 16 --epochs 120 --data /content/data.yaml --weights yolov5s.pt --name yolov5_results  --cache

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
train: weights=yolov5s.pt, cfg=, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=120, batch_size=16, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-95-gea72b84 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_g

In [ ]:
!pwd
# --save-txt for text output
# first 5 columns are:
# class, x_center, y_center, width, height

# --save-conf adds a 6th column along with --save-txt
# 6th column is confidence

# --nosave for not saving images


# !python detect.py --source {path/images} -- weights {path/best.pt} --name {string} -- save-txt

!python detect.py --source /content/test/images --weights /content/yolov5/runs/train/yolov5_results/weights/best.pt --name detect_test --save-txt

/content/yolov5
detect: weights=['/content/yolov5/runs/train/yolov5_results/weights/best.pt'], source=/content/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-95-gea72b84 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/50 /content/test/images/2022-01-09--100-_png.rf.7aa8fc9f1d158afe0d5f920a0fdfdb95.jpg: 384x640 1 bb8, Done. (0.027s)
image 2/50 /content/test/images/2022-01-09--103-_png.rf.15eb2c9019df40d6a7315490c36d4e25.jpg: 384x640 1 bb8, Done. (0.027s)
image 3/50 /content/test/images/2022-01-09--119-_png.rf.d7d118e65687bedcc0d

In [ ]:
# detections_dir = "/content/yolov5/runs/detect/detect_test3/"
# detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

# random_detection_image = PIL.Image.open(random.choice(detection_images))
# plt.imshow(np.array(random_detection_image))

In [ ]:
# !zip -(r)ecursive <dest.zip> <source>
!zip -r /content/drive/MyDrive/object_detection/960small_16batch_120epochs.zip /content/yolov5/runs/train/yolov5_results2/weights

  adding: content/yolov5/runs/train/yolov5_results2/weights/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5_results2/weights/best.pt (deflated 9%)
  adding: content/yolov5/runs/train/yolov5_results2/weights/last.pt (deflated 9%)


In [ ]:
# for downloading small files
# from google.colab import files
# files.download("/content/yolov5/runs/train/yolov5_results2")

In [ ]:
# shootout prep to compare different models
# not neccessary to actually train

# %%capture
!mkdir /content/shootout
!mkdir /content/shootout/weights
%cd /content/shootout
# download dataset from roboflow

!curl -L "https://app.roboflow.com/ds/{url}" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip #

%cd /content/yolov5

# UPLOAD WEIGHTS INTO APPROPRIATE DIRECTORY!

In [ ]:
# # shootout 320
# %cd /content/yolov5
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/1. 320 small best.pt' --name detect_run --save-conf --save-txt
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/2. 320 medium best.pt' --name detect_run --save-conf --save-txt
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/3. 320 large best.pt' --name detect_run --save-conf --save-txt


In [ ]:
# # shootout 960
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/4. 960 small best.pt' --name detect_run --save-conf --save-txt
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/5. 960 medium best.pt' --name detect_run --save-conf --save-txt

In [ ]:
# # bonus round 960 2x trained
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/6. 960 repeat.pt' --name detect_run --save-conf --save-txt

In [ ]:
# # shootout 960 30 vs 120 epochs
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/960small_30epochs_best.pt' --name detect_run --save-conf --save-txt
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/960small_120epochs_best.pt' --name detect_run --save-conf --save-txt

In [ ]:
# 120 epochs. 16 vs 32 batch
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/960img_16batch_120epoch_small.pt' --name detect_run --save-conf --save-txt
# !python detect.py --source /content/shootout/test/images --weights '/content/shootout/weights/960img_32batch_120epoch_small.pt' --name detect_run --save-conf --save-txt

detect: weights=['/content/shootout/weights/960img_16batch_120epoch_small.pt'], source=/content/shootout/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=detect_run, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/44 /content/shootout/test/images/2022-01-09--100-_png.rf.b7a18f9ca2c8332b54bbfb706253272a.jpg: 384x640 Done. (0.214s)
image 2/44 /content/shootout/test/images/2022-01-09--103-_png.rf.dbd1556c9cf386b99728834242f84d34.jpg: 384x640 1 bb8, Done. (0.224s)
image 3/44 /content/shootout/test/images/2022-01-09--119-_png.rf.7527a8ff5c9104c1d0dcb5e36665c77